In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc matplotlib numpy

# CHSH-Ungleichung

*Nutzungsschätzung: Zwee Minuten uff'n Heron r2-Prozessor (HINWEIS: Dit is bloß 'ne Schätzung. Deine Laufzeit kann variiern.)*
## Hintergrund
In dem Tutorial machen wa 'n Experiment uff'n Quantenrechner, umde Verletzung von de CHSH-Ungleichung mitn Estimator-Primitiv zu zeijen.

De CHSH-Ungleichung, benannt nach de Autoren Clauser, Horne, Shimony und Holt, wird jenutzt, um Bells Theorem (1969) experimentell zu beweisen. Dit Theorem sagt aus, datte lokale Theorien mit verborgene Variabln nich alle Konsequenzen von de Verschränkung in de Quantenmechanik erklären können. De Verletzung von de CHSH-Ungleichung wird jenutzt, um zu zeijen, datte Quantenmechanik mit lokale Theorien mit verborgene Variabln nich vereinbar is. Dit is'n wichtijet Experiment für't Verständnis von de Grundlagen von de Quantenmechanik.

De Nobelpreis für Physik 2022 wurde an Alain Aspect, John Clauser und Anton Zeilinger verjebn, unter anderm für ihre Pionierarbeit in de Quanteninformationswissenschaft und besondas für ihre Experimente mit verschränkte Photonen, de de Verletzung von de Bellschen Ungleichungen jzeigt habn.
## Anforderungen
Bevor du mit dem Tutorial anfängst, stell sicher, dasse Foljendes installiert hast:

* Qiskit SDK v1.0 oda neuer, mit [visualization](https://docs.quantum.ibm.com/api/qiskit/visualization)-Unterstützung
* Qiskit Runtime (`pip install qiskit-ibm-runtime`) v0.22 oda neuer
## Einrichtung

In [1]:
# General
import numpy as np

# Qiskit imports
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Qiskit Runtime imports
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import EstimatorV2 as Estimator

# Plotting routines
import matplotlib.pyplot as plt
import matplotlib.ticker as tck

## Schritt 1: Klassische Eingaben uff'n Quantenproblem abbildn
Für dit Experiment machen wa'n verschränktes Paar, wo wa jedes Qubit uff zwee verschiedene Basen messen. Wa kennzeichnen de Basen für't erste Qubit mit $A$ und $a$ und de Basen für't zweete Qubit mit $B$ und $b$. Dit erlaubt uns, de CHSH-Jröße $S_1$ zu berechnen:

$$
S_1 = A(B-b) + a(B+b).
$$

Jede Observable is entweder $+1$ oda $-1$. Klar is, datt eena von de Terme $B\pm b$ jleich $0$ sein muss und de andre $\pm 2$ sein muss. Deshalb is $S_1 = \pm 2$. De Durchschnittswert von $S_1$ muss de Ungleichung erfülln:

$$
|\langle S_1 \rangle|\leq 2.
$$

Wennwa $S_1$ in Bezug uff $A$, $a$, $B$ und $b$ auswickeln, kriejn wa:

$$
|\langle S_1 \rangle| = |\langle AB \rangle - \langle Ab \rangle + \langle aB \rangle + \langle ab \rangle| \leq 2
$$

Du kannste noch 'ne weitere CHSH-Jröße $S_2$ definiern:

$$
S_2 = A(B+b) - a(B-b),
$$

Dit führt zu 'ner weitern Ungleichung:

$$
|\langle S_2 \rangle| = |\langle AB \rangle + \langle Ab \rangle - \langle aB \rangle + \langle ab \rangle| \leq 2
$$

Wennde Quantenmechanik durch lokale Theorien mit verborgene Variabln beschriebn werdn kann, müssen de vorijen Ungleichungen wahr sein. Aba wie in dem Tutorial jezeigt wird, können diese Ungleichungen uff'n Quantenrechner verletzt werdn. Deshalb is de Quantenmechanik nich mit lokale Theorien mit verborgene Variabln vereinbar.
Fallste mehr Theorie lernen willst, juck dir [Entanglement in Action](/learning/courses/basics-of-quantum-information/entanglement-in-action/chsh-game) mitn John Watrous an.
Wa machen 'n verschränktes Paar zwischen zwee Qubits in 'n Quantenrechner, indemt wa'n Bell-Zustand $|\Phi^+\rangle = \frac{|00\rangle + |11\rangle}{\sqrt{2}}$ erzeugen. Mitn Estimator-Primitiv kannste direkt de nötijen Erwartungswerte ($\langle AB \rangle, \langle Ab \rangle, \langle aB \rangle$ und $\langle ab \rangle$) kriejn, um de Erwartungswerte von de beedn CHSH-Jrößen $\langle S_1\rangle$ und $\langle S_2\rangle$ zu berechnen. Vor de Einführung vom Estimator-Primitiv hätteste de Erwartungswerte aus de Messerjebnisse konstruiern müssen.

Wa messen dit zweete Qubit in de $Z$- und $X$-Basen. Dit erste Qubit wird ooch in orthogonale Basen jemessen, aba mit 'n Winkel bezüglich vom zweeten Qubit, den wa zwischen $0$ und $2\pi$ variiern werdn. Wie du sehn wirst, macht dit Estimator-Primitiv dit Ausführn von parametrisierten Schaltkreisen sehr einfach. Anstatt 'ne Reihe von CHSH-Schaltkreisen zu machen, musste bloß *een* CHSH-Schaltkreis mit 'n Parameter machen, de'n Messwinkel angibt, und 'ne Reihe von Phasenwerte für'n Parameter.

Schließlich werdn wa de Erjebnisse analysiern und jegen Messwinkel auftrahn. Du wirst sehn, datt für 'n bestimmten Bereich von Messwinkeln de Erwartungswerte von de CHSH-Jrößen $|\langle S_1\rangle| > 2$ oda $|\langle S_2\rangle| > 2$ sind, wat de Verletzung von de CHSH-Ungleichung demonstriert.

In [2]:
# To run on hardware, select the backend with the fewest number of jobs in the queue
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)
backend.name

'ibm_kingston'

### Create a parameterized CHSH circuit

First, we write the circuit with the parameter $\theta$, which we call `theta`. The [`Estimator` primitive](https://docs.quantum-computing.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2) can enormously simplify circuit building and output analysis by directly providing expectation values of observables. Many problems of interest, especially for near-term applications on noisy systems, can be formulated in terms of expectation values. `Estimator` (V2) primitive can automatically change measurement basis based on the supplied observable.

In [3]:
theta = Parameter("$\\theta$")

chsh_circuit = QuantumCircuit(2)
chsh_circuit.h(0)
chsh_circuit.cx(0, 1)
chsh_circuit.ry(theta, 0)
chsh_circuit.draw(output="mpl", idle_wires=False, style="iqp")

<Image src="../docs/images/tutorials/chsh-inequality/extracted-outputs/6c77e40a-0.avif" alt="Output of the previous code cell" />

### 'N parametrisierten CHSH-Schaltkreis machen
Erst schreibn wa'n Schaltkreis mitn Parameter $\theta$, den wa `theta` nennen. Dit [`Estimator`-Primitiv](https://docs.quantum-computing.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2) kann'n Schaltkreisaufbau und de Ausgabeanalyse jigantisch vereinfachen, indemmet direkt Erwartungswerte von Observabeln liefert. Ville interessante Probleme, besondas für kurzfristije Anwendungen uff verrauschte Systeme, können in Form von Erwartungswerte formuliert werdn. Dit `Estimator` (V2)-Primitiv kann automatisch de Messbasis basiernd uff de bereitjestellte Observable ändern.

In [4]:
number_of_phases = 21
phases = np.linspace(0, 2 * np.pi, number_of_phases)
# Phases need to be expressed as list of lists in order to work
individual_phases = [[ph] for ph in phases]

![Output of the previous code cell](../docs/images/tutorials/chsh-inequality/extracted-outputs/6c77e40a-0.avif)

### 'Ne Liste von Phasenwerte machen, de späta zujewiesen werdn
Nachdem du'n parametrisierten CHSH-Schaltkreis jemacht hast, machste 'ne Liste von Phasenwerte, de dem Schaltkreis im nächsten Schritt zujewiesen werdn. Du kannst'n folgenden Code nutzen, um 'ne Liste von 21 Phasenwerte im Bereich von $0$ bis $2 \pi$ mit jleichem Abstand zu machen, also $0$, $0.1 \pi$, $0.2 \pi$, ..., $1.9 \pi$, $2 \pi$.

In [5]:
# <CHSH1> = <AB> - <Ab> + <aB> + <ab> -> <ZZ> - <ZX> + <XZ> + <XX>
observable1 = SparsePauliOp.from_list(
    [("ZZ", 1), ("ZX", -1), ("XZ", 1), ("XX", 1)]
)

# <CHSH2> = <AB> + <Ab> - <aB> + <ab> -> <ZZ> + <ZX> - <XZ> + <XX>
observable2 = SparsePauliOp.from_list(
    [("ZZ", 1), ("ZX", 1), ("XZ", -1), ("XX", 1)]
)

### Observabeln
Jetzt brauchen wa Observabeln, aus dene wa de Erwartungswerte berechnen können. In unserm Fall betrachtm wa orthogonale Basen für jedes Qubit, wobei de parametrisierte $Y$-Rotation für't erste Qubit de Messbasis beinah kontinuierlich bezüglich von de Basis vom zweeten Qubit variiert. Wa wähln deshalb de Observabeln $ZZ$, $ZX$, $XZ$ und $XX$.

In [6]:
target = backend.target
pm = generate_preset_pass_manager(target=target, optimization_level=3)

chsh_isa_circuit = pm.run(chsh_circuit)
chsh_isa_circuit.draw(output="mpl", idle_wires=False, style="iqp")

<Image src="../docs/images/tutorials/chsh-inequality/extracted-outputs/9a5561eb-0.avif" alt="Output of the previous code cell" />

## Schritt 2: Problem für de Ausführung uff Quantenhardware optimiern

Um de Jesamtausführungszeit vom Job zu reduzieren, akzeptiern V2-Primitive bloß Schaltkreise und Observabeln, de de vom Zielsystem unterstützten Anweisungen und de Konnektivität entsprechn (bezeichnet als Instruction Set Architecture (ISA)-Schaltkreise und -Observabeln).

### ISA-Schaltkreis

In [7]:
isa_observable1 = observable1.apply_layout(layout=chsh_isa_circuit.layout)
isa_observable2 = observable2.apply_layout(layout=chsh_isa_circuit.layout)

![Output of the previous code cell](../docs/images/tutorials/chsh-inequality/extracted-outputs/9a5561eb-0.avif)

### ISA-Observabeln

Ebenfalls müssen wa de Observabeln transformiern, umse backend-kompatibel zu machen, bevor wa Jobs mit [`Runtime Estimator V2`](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/estimator-v2#run) ausführn. Wa können de Transformation mit de `apply_layout`-Methode vom `SparsePauliOp`-Objekt durchführn.

In [8]:
# To run on a local simulator:
# Use the StatevectorEstimator from qiskit.primitives instead.

estimator = Estimator(mode=backend)

pub = (
    chsh_isa_circuit,  # ISA circuit
    [[isa_observable1], [isa_observable2]],  # ISA Observables
    individual_phases,  # Parameter values
)

job_result = estimator.run(pubs=[pub]).result()

## Schritt 3: Ausführn mit Qiskit-Primitiven
Um dit jesamte Experiment in 'n einzijen Aufruf vom [`Estimator`](https://docs.quantum-computing.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2) auszuführn.
Wa können 'n [Qiskit Runtime `Estimator`](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/estimator-v2)-Primitiv machen, um unsre Erwartungswerte zu berechnen. De `EstimatorV2.run()`-Methode nimmt 'n Iterable von `primitive unified blocs (PUBs)`. Jedes PUB is 'n Iterable im Format `(circuit, observables, parameter_values: Optional, precision: Optional)`.

In [9]:
chsh1_est = job_result[0].data.evs[0]
chsh2_est = job_result[0].data.evs[1]

In [10]:
fig, ax = plt.subplots(figsize=(10, 6))

# results from hardware
ax.plot(phases / np.pi, chsh1_est, "o-", label="CHSH1", zorder=3)
ax.plot(phases / np.pi, chsh2_est, "o-", label="CHSH2", zorder=3)

# classical bound +-2
ax.axhline(y=2, color="0.9", linestyle="--")
ax.axhline(y=-2, color="0.9", linestyle="--")

# quantum bound, +-2√2
ax.axhline(y=np.sqrt(2) * 2, color="0.9", linestyle="-.")
ax.axhline(y=-np.sqrt(2) * 2, color="0.9", linestyle="-.")
ax.fill_between(phases / np.pi, 2, 2 * np.sqrt(2), color="0.6", alpha=0.7)
ax.fill_between(phases / np.pi, -2, -2 * np.sqrt(2), color="0.6", alpha=0.7)

# set x tick labels to the unit of pi
ax.xaxis.set_major_formatter(tck.FormatStrFormatter("%g $\\pi$"))
ax.xaxis.set_major_locator(tck.MultipleLocator(base=0.5))

# set labels, and legend
plt.xlabel("Theta")
plt.ylabel("CHSH witness")
plt.legend()
plt.show()

<Image src="../docs/images/tutorials/chsh-inequality/extracted-outputs/f6267448-0.avif" alt="Output of the previous code cell" />

In the figure, the lines and gray areas delimit the bounds; the outer-most (dash-dotted) lines delimit the quantum-bounds ($\pm 2$), whereas the inner (dashed) lines delimit the classical bounds ($\pm 2\sqrt{2}$). You can see that there are regions where the CHSH witness quantities exceeds the classical bounds. Congratulations! You have successfully demonstrated the violation of CHSH inequality in a real quantum system!

## Tutorial survey

Please take this short survey to provide feedback on this tutorial. Your insights will help us improve our content offerings and user experience.

[Link to survey](https://your.feedback.ibm.com/jfe/form/SV_3xxAgm1SF1wGp9k)